In [1]:
import os
import nd2
import napari
import pickle
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.measure import regionprops_table, profile_line
from cardiomyocytes_helper_functions import segment_actin_3D,calculate_orientation,find_fibers_orientation_v2,orientations_from_vertices, signal_from_vertices,divide_cell_outside_ring

In [2]:
path_dir = r'D:\data_analysis\2022_Sahana\data\Collagen\60x images'
im_name = r'092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_001.nd2'
path_im = os.path.join(path_dir,im_name)

path_vertices = r'D:\data_analysis\2022_Sahana\masks'

path_results = r'D:\data_analysis\2022_Sahana\results'
path_df = os.path.join(path_results,im_name.replace('.nd2','_df.pkl'))

# order of channels
    # actin
    # paxilin
    # plakoglobin
    # DAPI


## Read in the image and annotations

In [3]:
# read in image
im = nd2.imread(path_im)
im.shape

(27, 4, 512, 512)

In [4]:
# read in polygons and masks

pkl_path = os.path.join(path_vertices,im_name.replace('.nd2','_polygons.pkl'))

with open(pkl_path, 'rb') as f:
    vertices_polygons = pickle.load(f)

mask_path = os.path.join(path_vertices,im_name.replace('.nd2','_mask.png'))
mask = imread(mask_path)

## Segment actin

In [5]:
# get actin channel
image_actin = im[:,0,:,:]

# segment actin volume
image_actin_mask = segment_actin_3D(image_actin)

# flatten segmented actin
image_actin_mask_2D = np.max(image_actin_mask,axis=0)

intensity normalization: min-max normalization with NO absoluteintensity upper bound


## Calculate general properties of cells

In [6]:
# calculate properties of cells

properties = ['label','area','centroid','bbox','eccentricity','orientation','intensity_image','image']

im_flat_all_channels = np.max(im,axis=0)
im_flat_all = np.append(im_flat_all_channels,np.expand_dims(image_actin_mask_2D,axis=0),axis=0)
im_flat_all = np.moveaxis(im_flat_all,0,2)

cell_measure = regionprops_table(mask, intensity_image = im_flat_all, properties = properties)
df = pd.DataFrame(cell_measure)
df['image_name'] = im_name

In [7]:
# add vertices to the table
df['vertices'] = vertices_polygons

## Characterize orientation of the cell edge (based on vertices)

In [8]:
# characterize the orientation of the edge of the cell

df['membrane_orientation'] = None
df['membrane_orientation'] = df['membrane_orientation'].astype(object)

for i,cell in df.iterrows():

    vert = cell.vertices

    membrane_orientation = orientations_from_vertices(vert)

    df.at[i,'membrane_orientation'] = membrane_orientation


## Calculate general flow of actin

In [9]:
# characterize actin orientation in the cells

df['actin_detected'] = None
df['actin_detected'] = df['actin_detected'].astype(object)
df['actin_angles'] = None
df['actin_angles'] = df['actin_angles'].astype(object)

for i,cell in df.iterrows():

    im_single_cell_actin = cell.intensity_image[:,:,4]

    actin,actin_angles = find_fibers_orientation_v2(im_single_cell_actin)
    
    # calculate mean orientation of the fibers
    actin_orientation = np.mean(actin_angles)

    # calculate spread of orientations
    actin_spread = np.std(actin_angles)

    df.at[i,'actin_detected'] = np.array(actin)
    df.at[i,'actin_angles'] = np.array(actin_angles)
    df.loc[i,'actin_orientation'] = actin_orientation
    df.loc[i,'actin_spread'] = actin_spread

## Characterize signals at the edge

In [10]:
signal_max = np.max(im[:,2,:,:],axis=0)

df['plak_signal'] = None
df['plak_signal'] = df['plak_signal'].astype(object)

for i,cell in df.iterrows():

    vert = cell.vertices

    signal_line = signal_from_vertices(vert,signal_max)

    df.at[i,'plak_signal'] = signal_line

## Generate edge regions

In [12]:
ring_thickness = 10
segment_number = 25

df['outside_ring_regions'] = None

for i,cell in df.iterrows():

    cell_image = cell.image

    points_array = divide_cell_outside_ring(cell_image,ring_thickness,segment_number)

    df.at[i,'outside_ring_regions'] = points_array

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than

## Visualize image

In [11]:
viewer = napari.Viewer()
viewer.add_image(im[:,0,:,:],blending='additive',colormap='magenta')
viewer.add_image(signal_max,blending='additive',colormap='green')
viewer.add_image(image_actin_mask,blending='additive',colormap='green',visible=False)
viewer.add_image(image_actin_mask_2D,blending='additive',colormap='gray')
viewer.add_labels(mask,blending='additive')

Assistant skips harvesting pyclesperanto as it's not installed.


<Labels layer 'mask' at 0x26d9934ce20>

## Show data frame

In [13]:
df

,label,area,centroid-0,centroid-1,bbox-0,bbox-1,bbox-2,bbox-3,eccentricity,orientation,...,image,image_name,vertices,membrane_orientation,actin_detected,actin_angles,actin_orientation,actin_spread,plak_signal,outside_ring_regions
0,1,12222,417.672640,305.914662,356,193,492,425,0.950820,-1.178678,...,"[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...,"[[413.5046444321612, 203.5532564629244], [461....","[-0.2317168411904511, 0.7044940642422182, 1.41...","[[[180, 73], [153, 70]], [[118, 26], [138, 11]...","[1.4601391056210011, -0.9272952180016122, -0.6...",-1.315784,1.097547,"[362.0, 341.0, 337.0, 401.0, 554.0, 889.0, 151...","[[0, 183, 15], [0, 184, 15], [0, 185, 15], [0,..."
1,2,5864,416.320259,287.201739,371,240,468,337,0.635507,0.802674,...,"[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...,"[[391.8601720840474, 239.80774764601492], [410...","[0.14601225771127702, 0.5266272714337505, 0.61...","[[[62, 70], [80, 58]], [[32, 51], [49, 44]], [...","[-0.982793723247329, -1.1801892830972098, -1.0...",-1.006116,0.122593,"[1742.0, 1862.0, 1903.0, 1424.0, 830.0, 528.0,...","[[0, 32, 21], [0, 33, 21], [0, 34, 21], [1, 31..."
2,3,3475,304.857842,29.730360,243,9,360,57,0.948991,0.127392,...,"[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...,"[[251.71221363001072, 8.753005329900446], [279...","[0.17467219900823938, -0.057628427477474986, 0...","[[[13, 22], [10, 7]], [[44, 93], [39, 71]], [[...","[0.19739555984988089, 0.22347660114063306, 0.2...",0.260687,0.084831,"[388.0, 399.0, 390.0, 420.0, 427.0, 390.0, 354...","[[0, 18, 7], [1, 16, 7], [1, 17, 7], [1, 18, 7..."
3,4,1829,378.172772,55.621651,355,16,403,97,0.957310,-1.224663,...,"[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...,"[[379.95571229258485, 15.246347034334576], [40...","[0.2782996590051119, -1.2231894670389956, 0.26...","[[[59, 27], [52, 8]], [[30, 35], [24, 19]], [[...","[0.3529903878269105, 0.3587706702705722, 0.339...",0.245671,0.115946,"[257.0, 245.0, 256.0, 273.0, 265.0, 242.0, 258...","[[0, 72, 0], [0, 73, 0], [0, 74, 0], [1, 69, 0..."
4,5,5126,191.244635,186.167577,135,148,237,234,0.875291,0.608647,...,"[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...,"[[157.5587589157158, 149.9831874013428], [191....","[0.11065722117389543, 0.4423742229767451, 0.99...","[[[27, 53], [23, 38]], [[52, 57], [36, 27]], [...","[0.260602391747341, 0.48995732625372845, 0.479...",0.417871,0.160449,"[241.0, 231.0, 237.0, 257.0, 290.0, 313.0, 297...","[[0, 7, 0], [0, 8, 0], [0, 9, 0], [0, 10, 0], ..."
5,6,4440,49.332432,250.242793,12,198,87,302,0.799329,1.142257,...,"[[False, False, False, False, False, False, Fa...",092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-...,"[[10.917458757245019, 228.4443996632552], [24....","[-0.7437555842988597, -0.7313966630214805, 0.9...","[[[75, 44], [53, 31]], [[87, 44], [52, 16]], [...","[1.037088135155635, 0.8960553845713441, 1.0851...",1.065597,0.099889,"[259.0, 302.0, 283.0, 223.0, 215.0, 231.0, 242...","[[0, 30, 8], [0, 31, 8], [0, 32, 8], [0, 33, 8..."


## Save data frame

In [14]:
df.to_pickle(path_df)